# Нарезка документов при помощи токенизатора

В данном юпитер-блокноте мы рассмотрим как можно при помощи библиотеки LangChain нарезать на чанки при помощи комбинации из посимвольной нарезки и токенизаторов.  

## Решение без сторонних библиотек

В данном примере используется текст Конституции Российской Федерации.

In [6]:
file_path = 'constitutionrf.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

## Нарезка при помощи токенизатора TikToken

Подгрузим класс `CharacterTextSplitter`, но инициализацию выполним при помощи метода `from_tiktoken_encoder`, данный метод сделает вызов метода `encode` из пакета `tiktoken`, определи eos и bos токены, после чего попытается нарезать документ таким образом, чтобы размер чанков не выходил за указанные пределы.  

In [7]:
from langchain_text_splitters import CharacterTextSplitter
from pprint import pprint

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    #encoding=get_encoding('cl100k_base'),  # В новых версия LangChain эту опция более недоступна
    chunk_size=200,
    chunk_overlap=20
)
chunks = text_splitter.split_text(text)

print(f"Всего чанков: {len(chunks)}")
print("Первые N чанков:")
chunks[:10]

Created a chunk of size 787, which is longer than the specified 200
Created a chunk of size 514, which is longer than the specified 200
Created a chunk of size 301, which is longer than the specified 200
Created a chunk of size 919, which is longer than the specified 200
Created a chunk of size 461, which is longer than the specified 200
Created a chunk of size 557, which is longer than the specified 200
Created a chunk of size 354, which is longer than the specified 200
Created a chunk of size 315, which is longer than the specified 200
Created a chunk of size 214, which is longer than the specified 200
Created a chunk of size 675, which is longer than the specified 200
Created a chunk of size 237, which is longer than the specified 200
Created a chunk of size 674, which is longer than the specified 200
Created a chunk of size 215, which is longer than the specified 200
Created a chunk of size 1078, which is longer than the specified 200
Created a chunk of size 333, which is longer th

Всего чанков: 273
Первые N чанков:


['\ufeffКОНСТИТУЦИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ',
 'Мы, многонациональный народ Российской Федерации,\nсоединенные общей судьбой на своей земле,\nутверждая права и свободы человека, гражданский мир и согласие,\nсохраняя исторически сложившееся государственное единство,\nисходя из общепризнанных принципов равноправия и самоопределения народов,\nчтя память предков, передавших нам любовь и уважение к Отечеству, веру в добро и справедливость,\nвозрождая суверенную государственность России и утверждая незыблемость ее демократической основы,\nстремясь обеспечить благополучие и процветание России,\nисходя из ответственности за свою Родину перед нынешним и будущими поколениями,\nсознавая себя частью мирового сообщества,\nпринимаем КОНСТИТУЦИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.',
 'РАЗДЕЛ ПЕРВЫЙ\n\nГЛАВА 1.\nОСНОВЫ КОНСТИТУЦИОННОГО СТРОЯ\n\nСтатья 1',
 'Статья 1\n\n1. Российская Федерация - Россия есть демократическое федеративное правовое государство с республиканской формой правления.\n2. Наименования Российская

## Рекурсивная нарезка с помощью токенизатора TikToken 

В данном примере мы используем класс рекурсивной нарезки, однако, теперь на последней миле нарезка будет выполняться на токенизированном тексте.

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    model_name="gpt-4",
    chunk_size=200,
    chunk_overlap=20
)

chunks = text_splitter.split_text(text)

print(f"Всего чанков: {len(chunks)}")
print("Первые N чанков:")
chunks[:10]

Всего чанков: 463
Первые N чанков:


['\ufeffКОНСТИТУЦИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ',
 'Мы, многонациональный народ Российской Федерации,\nсоединенные общей судьбой на своей земле,\nутверждая права и свободы человека, гражданский мир и согласие,\nсохраняя исторически сложившееся государственное единство,\nисходя из общепризнанных принципов равноправия и самоопределения народов,\nчтя память предков, передавших нам любовь и уважение к Отечеству, веру в добро и справедливость,',
 'возрождая суверенную государственность России и утверждая незыблемость ее демократической основы,\nстремясь обеспечить благополучие и процветание России,\nисходя из ответственности за свою Родину перед нынешним и будущими поколениями,\nсознавая себя частью мирового сообщества,\nпринимаем КОНСТИТУЦИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.',
 'РАЗДЕЛ ПЕРВЫЙ\n\nГЛАВА 1.\nОСНОВЫ КОНСТИТУЦИОННОГО СТРОЯ\n\nСтатья 1\n\n1. Российская Федерация - Россия есть демократическое федеративное правовое государство с республиканской формой правления.\n2. Наименования Российская Федерация

## Нарезка при помощи SentenceTransformers

В данном примере задействован токенизаторы использующий модели типа SentenceTransformer.

In [9]:
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

splitter = SentenceTransformersTokenTextSplitter(
    model_name="sentence-transformers/all-mpnet-base-v2",
    chunk_size=200, chunk_overlap=20
)
text_token_count = splitter.count_tokens(text=text) - 2
token_multiplier = splitter.maximum_tokens_per_chunk // text_token_count + 1
text_to_split = text * token_multiplier  # Пример использования для нарезки большого текста
print(f"Токенов в тексте для нарезки: {splitter.count_tokens(text=text_to_split)}")

chunks = splitter.split_text(text=text_to_split)

print(f"Всего чанков: {len(chunks)}")
print("Первые N чанков:")
chunks[:10]

/home/pasha/Documents/Repository/gpt/text-splitting/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Токенов в тексте для нарезки: 105819
Всего чанков: 291
Первые N чанков:


['конституция россиискои федерации мы, многонациональныи народ россиискои федерации, соединенные общеи судьбои на своеи земле, утверждая права и свободы человека, гражданскии мир и согласие, сохраняя исторически сложившееся государственное единство, исходя из общепризнанных принципов равноправия и самоопределения народов, чтя память предков, передавших нам любовь и уважение к отечеству, веру в добро и справедливость, возрождая суверенную государственност',
 '##нную государственность россии и утверждая незыблемость ее демократическои основы, стремясь обеспечить благополучие и процветание россии, исходя из ответственности за свою родину перед нынешним и будущими поколениями, сознавая себя частью мирового сообщества, принимаем конституцию россиискои федерации. раздел первыи глава 1. основы конституционного строя статья 1 1. россииская федерация - россия есть демократическое федеративное правовое государство',
 '##ое правовое государство с республиканскои формои правления. 2. наименования 

## Нарезка текста при помощи токенизатора моделей с HuggingFace

В данном примере мы у класс `CharacterTextSplitter` делаем вызов метода `from_huggingface_tokenizer` котором на вход передаём объект токенизатора из пакета trahsformers, что позволяет нам использовать для нарезки текста любой токенизатор любой модели.  

In [10]:
from transformers import GPT2TokenizerFast
from langchain_text_splitters import CharacterTextSplitter

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer, chunk_size=100, chunk_overlap=0
)

chunks = text_splitter.split_text(text)

print(f"Всего чанков: {len(chunks)}")
print("Первые N чанков:")
chunks[:10]

Created a chunk of size 787, which is longer than the specified 100
Created a chunk of size 191, which is longer than the specified 100
Created a chunk of size 162, which is longer than the specified 100
Created a chunk of size 514, which is longer than the specified 100
Created a chunk of size 301, which is longer than the specified 100
Created a chunk of size 919, which is longer than the specified 100
Created a chunk of size 461, which is longer than the specified 100
Created a chunk of size 557, which is longer than the specified 100
Created a chunk of size 354, which is longer than the specified 100
Created a chunk of size 315, which is longer than the specified 100
Created a chunk of size 214, which is longer than the specified 100
Created a chunk of size 675, which is longer than the specified 100
Created a chunk of size 237, which is longer than the specified 100
Created a chunk of size 674, which is longer than the specified 100
Created a chunk of size 215, which is longer tha

Всего чанков: 293
Первые N чанков:


['\ufeffКОНСТИТУЦИЯ РОССИЙСКОЙ ФЕДЕРАЦИИ',
 'Мы, многонациональный народ Российской Федерации,\nсоединенные общей судьбой на своей земле,\nутверждая права и свободы человека, гражданский мир и согласие,\nсохраняя исторически сложившееся государственное единство,\nисходя из общепризнанных принципов равноправия и самоопределения народов,\nчтя память предков, передавших нам любовь и уважение к Отечеству, веру в добро и справедливость,\nвозрождая суверенную государственность России и утверждая незыблемость ее демократической основы,\nстремясь обеспечить благополучие и процветание России,\nисходя из ответственности за свою Родину перед нынешним и будущими поколениями,\nсознавая себя частью мирового сообщества,\nпринимаем КОНСТИТУЦИЮ РОССИЙСКОЙ ФЕДЕРАЦИИ.',
 'РАЗДЕЛ ПЕРВЫЙ\n\nГЛАВА 1.\nОСНОВЫ КОНСТИТУЦИОННОГО СТРОЯ',
 'Статья 1',
 '1. Российская Федерация - Россия есть демократическое федеративное правовое государство с республиканской формой правления.\n2. Наименования Российская Федерация 